In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
puntajes = pd.read_csv("../A_INSCRITOS_PUNTAJES_2023_PAES_PUB_MRUN.csv", sep=";")
puntajes.head()

In [ ]:
socioeconomic = pd.read_csv("../B_SOCIOECONOMICO_DOMICILIO_2023_PAES_PUB_MRUN.csv", sep=";")
socioeconomic.head()

In [ ]:
new_1 = socioeconomic.drop(['FECHA_NACIMIENTO','ANYO_PROCESO', 'CODIGO_REGION_DOMICILIO','CODIGO_PROVINCIA_DOMICILIO','CODIGO_COMUNA_DOMICILIO','NOMBRE_COMUNA_DOMICILIO'], axis=1)
#quite ciertos datos de socioeconomic que no ibamos a usar :p

In [ ]:
columnas_puntajes = ['MRUN', 'DEPENDENCIA', 'PROMEDIO_NOTAS','PTJE_NEM','PTJE_RANKING','CLEC_REG_ACTUAL','MATE1_REG_ACTUAL','MATE2_REG_ACTUAL','HCSOC_REG_ACTUAL','CIEN_REG_ACTUAL','PROMEDIO_CM_MAX','CLEC_MAX','MATE1_MAX','MATE2_MAX','HCSOC_MAX','CIEN_MAX']
new_2 = puntajes[columnas_puntajes]
#quite unos datos de puntajes tambien

In [ ]:
#merge entre new_1 y new_2
merged_new = pd.merge(new_1,new_2,on='MRUN')
merged_new

Aquí empieza la limpieza de datos de las columnas de merged_new

In [ ]:
tipos_dato = merged_new.dtypes 
print(tipos_dato)

In [ ]:
import numpy as np

#cambiar las comas por puntos
merged_new['PROMEDIO_CM_MAX'] = merged_new['PROMEDIO_CM_MAX'].str.replace(',', '.').astype(float)
merged_new['PROMEDIO_NOTAS'] = merged_new['PROMEDIO_NOTAS'].str.replace(',', '.').astype(float)

#para todas las columnas que poseen N o S se cambia por 0 o 1
cols = ['DISPOSITIVO_1', 'DISPOSITIVO_2', 'DISPOSITIVO_3', 'DISPOSITIVO_4', 'DISPOSITIVO_5', 'HOGAR_CONEXION_INTERNET', 'ESPACIO_1', 'ESPACIO_2', 'ESPACIO_3', 'ESPACIO_4', 'ESPACIO_5', 'ESPACIO_6', 'ESPACIO_7', 'ESPACIO_8', 'ESPACIO_9']

# Valores distintos a 'N' o 'S' se reemplazan por NaN
for col in cols:
    merged_new[col] = merged_new[col].apply(lambda x: x if x in ['N', 'S'] else np.nan)

# Eliminar los NaN
merged_new.dropna(subset=cols, inplace=True)

# Transformación de tipo
for col in cols:
    merged_new[col] = merged_new[col].str.replace('N', '0').str.replace('S', '1').astype(int)
    print(f"Columna transformada: {col}")

print("\nDataFrame después de la transformación:")
print(merged_new)

In [ ]:
#para los datos Nan que son discretos se cambian a 0, quizas despues puede ser el promedio 
for col in merged_new.select_dtypes(include=['object']).columns:
    merged_new[col] = merged_new[col].replace(' ', '0')
    merged_new[col] = merged_new[col].astype(int)
    print(f"Columna transformada: {col}")

Clusters con KMeans

In [ ]:
#seleccion de columnas para hacer el cluster 
# data1 = merged_new[['PROMEDIO_CM_MAX','RAZON_PRINCIPAL_PAES','USO_DISPOSITIVO_1','USO_DISPOSITIVO_2','USO_DISPOSITIVO_3','CONEXION_INSUFICIENTE','USO_ESPACIO_1','USO_ESPACIO_2','USO_ESPACIO_3','USO_ESPACIO_4','USO_ESPACIO_5','USO_ESPACIO_6','USO_ESPACIO_7','USO_ESPACIO_8','TRANQUILIDAD_COLEGIO','TRABAJO_GRUPO_COLEGIO']]
data1 = merged_new[['PROMEDIO_CM_MAX','RAZON_PRINCIPAL_PAES','USO_DISPOSITIVO_1','USO_DISPOSITIVO_2','USO_DISPOSITIVO_3','CONEXION_INSUFICIENTE','USO_ESPACIO_1','USO_ESPACIO_2','USO_ESPACIO_3','USO_ESPACIO_4','USO_ESPACIO_5','USO_ESPACIO_6','USO_ESPACIO_7','USO_ESPACIO_8','TRANQUILIDAD_COLEGIO','TRABAJO_GRUPO_COLEGIO']]
idxs = [1, 4, 7, 13]
data = data1[(data1['RAZON_PRINCIPAL_PAES'].isin([1,2,3,4,5,6,8]))& (data1['PROMEDIO_CM_MAX']>=200)]
data

In [ ]:
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import GenericUnivariateSelect, f_classif, f_regression

#se normalizan los datos
normalizer = Normalizer()
scaled_data = normalizer.fit_transform(data)

#se eligen los atributos que se utilizaran para hacer el cluster usando genericunivariateselect, de ahi lo podemos cambiar por otro, esto era para ver si funcionaba xd
y = data['PROMEDIO_CM_MAX']
#se supone que al usar f_regression utiliza la correlacion de pearson para determinar que caracteristicas tienen mayor correlacion con promedio_max,
#selecciona el 50% de caracteristicas con mayor correlacion
#puede usarse percentil o kbest, pero kbest uno determina con un valor fijo
selector = GenericUnivariateSelect(score_func=f_regression, mode='percentile', param=50)
selector.fit(scaled_data, y)
#atributos elegidos
selected_features = selector.transform(scaled_data)

support = selector.get_support()
selected_indices = np.where(support)[0]
#nombre de los atributos elegidos
selected_feature_names = data.columns[selected_indices]
selected_feature_names

In [ ]:
#revisar cuantos clusters crear
#inertia es SSE
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline
means = []
inertias = []
def optimize_k_means(data,max_k):
    for k in range(1,max_k):
        k_means = KMeans(n_clusters=k)
        k_means.fit(data)
        means.append(k)
        inertias.append(k_means.inertia_)

    fig = plt.subplots(figsize=(5,5))
    plt.plot(means,inertias, 'o-')
    plt.xlabel('Number Clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    plt.show()

optimize_k_means(scaled_data,10)

In [ ]:
#se hacen dos clusters de kmeans para probar
from sklearn.decomposition import PCA

kmeans = KMeans(n_clusters=2) #se toma el numero de clusters dependiendo del grafico de codo y se hace el kmeans clustering
kmeans.fit(scaled_data)
labels = kmeans.labels_

y_pca = scaled_data[:, 0]

columnas = np.take(scaled_data, idxs, axis=1)
pca2 = PCA(n_components=1, random_state=0)
x_pca = pca2.fit_transform(columnas)

#se visualiza el grafico
plt.scatter(x_pca, y_pca, c=labels, cmap='viridis')
plt.title('Clustering K-means con PCA')
plt.xlabel('Variables espacios de estudio')
plt.ylabel('Puntaje')
plt.show()